In [1]:
from pdf2image import convert_from_path
import pytesseract 
import cv2
import numpy as np
import re
from PIL import Image

In [2]:
pages = convert_from_path('../resources/patient_details/pd_1.pdf', poppler_path='C:/poppler-24.07.0/Library/bin')

In [3]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398>]

In [4]:
pages[0].show()

In [5]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
text = pytesseract.image_to_string(pages[0], lang='eng')
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



In [6]:
def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        61,
        11
    )
    return processed_image

In [7]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()

text = pytesseract.image_to_string(img, lang='eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight’
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Casc of Emergency
7 ee
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone

Genera! Medical History

a

a

a ea A CE i a

Chicken Pox (Varicella): Measies:

IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?
No

List any Medical Problems (asthma, seizures, headaches}:

Migraine

CO
aa



<h4>Extract Name</h4>

In [29]:
pattern = r'Patient Information(.*?)\(\d{3}\)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0]

' Birth Date\n\nJerry Lucas May 2 1998\n\n'

In [30]:
matches[0].strip()

'Birth Date\n\nJerry Lucas May 2 1998'

In [31]:
match = matches[0].replace("Birth Date", " ").strip()
match

'Jerry Lucas May 2 1998'

In [32]:
pattern = r'((Jan|Feb|Mar|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
date_matches = re.findall(pattern, text)
date = date_matches[0][0]
date

'May 2 1998'

In [33]:
match.replace(date, ' ').strip()

'Jerry Lucas'

In [34]:
def remove_noise_from_name(name):
    name = name.replace("Birth Date", " ").strip()
    date_pattern = r'((Jan|Feb|Mar|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(pattern, text)

    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date, ' ').strip()
    return name

In [35]:
name = 'Birth Date\n\nJerry Lucas May 2 1998\n\n'
name = remove_noise_from_name(name)
name

'Jerry Lucas'

<h4>Extract Phone Number</h4>

In [36]:
pattern = r'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[(' Birth Date\n\nJerry Lucas May 2 1998\n\n', '(279) 920-8204')]

In [37]:
matches[0][1]

'(279) 920-8204'

<h4>Extract Vaccine</h4>

In [38]:
pattern = r'Have you had the Hepatitis B vaccination\?.*(Yes|No)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['Yes']

<h4>Medical Problems</h4>

In [42]:
pattern = r'List any Medical Problems \(asthma, seizures, headaches\):(.*)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'N/A\n\n7?\nv\n\n17/12/2020'